In [1]:
import json
from konlpy.tag import Kkma
from konlpy.utils import pprint
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC,LinearSVC,SVR
from sklearn.metrics import accuracy_score,mean_squared_error,roc_auc_score,f1_score
import numpy as np
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn import grid_search, cross_validation

C:\Users\kvtor\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\kvtor\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
REVIEWS = "cs_reviews"
POS = "nouns"

In [3]:
f = open("../reviews_in_one_txt/" + REVIEWS + ".txt","r",encoding="utf-8").readlines()

In [4]:
g = open("../reviews_in_one_txt/" + REVIEWS + "_" + POS + ".txt","r",encoding="utf-8").readlines()

In [5]:
# kkma = Kkma()
# with open("../reviews_in_one_txt/" + REVIEx`WS+"_nouns.txt","w",encoding="utf-8") as out:
#     for line in f:
#         course = json.loads(line)
#         nouns = kkma.nouns(course["review_text"])
# #         print(nouns)
#         out.write(" ".join(nouns)+"\n")
        

In [6]:
mp = {"A":5,"B":4,"C":3,"D":2,"F":1}
# mp = {"A":1,"B":1,"C":0,"D":0,"F":0}

In [ ]:
count = {}
for i in range(len(f)):
    course = json.loads(f[i])
#     print(course)
    if course["강의"] != "?" and len(g[i])>1:
        if mp[course["강의"]] not in count:
            count[mp[course["강의"]]] = 0
        count[mp[course["강의"]]] += 1
        
print(count)

In [12]:
upper_limit = min(count.values())
upper_limit *= 200 
print(upper_limit)

7000


In [13]:
concat_nouns_list = []
for k,v in count.items():
    count[k] = 0

labels = []
important_pos = ["NNG","MAG","VV","OL","VA"]
for i in range(len(f)):
    line = f[i]
    course = json.loads(line)
    if course["강의"] == "?" or len(g[i])<=1:
        continue
    pos_list = g[i].split()
#     print(pos_list)
#     l = [x[1:-1].split(',') for x in pos_list]
#     print(l[:2])
#     to_be_joined = []
#     for p in l:
#         if p[1] in important_pos:
#             to_be_joined.append(p[0])
#     for p in l:
#         if p[1] in important_pos:
#             to_be_joined.append(p[0])
    to_be_joined = pos_list
    concat_nouns = " ".join(to_be_joined)
    if count[mp[course["강의"]]] < upper_limit:
        concat_nouns_list.append(concat_nouns)
#         labels.append(int(mp[course["강의"]]))
        labels.append(i)
        count[mp[course["강의"]]] += 1

In [14]:
labels = np.array(labels)
concat_nouns_list = np.array(concat_nouns_list)
data_train, data_test, y_train, y_test = train_test_split(concat_nouns_list,
                                    labels,test_size=0.1,random_state=42)

In [15]:
len(data_train)

432

In [16]:
len(y_train)

432

In [17]:
list(data_test)[3]

'저 이동 교수님 정보 수강 수강신청 신청 때 걱정 필요 강의 열의 수 영어 진행 말씀 제가 널 널널 떡 하면 악명 과제 프로젝트 켄즈 3 3번 번 포함 총 5 5개 개 난이도 생각 메뉴 메뉴얼 얼 시작 야 야매 매 테스트 통과 안 성적 주실 거'

In [18]:
y_train[2]

676

In [19]:
def parse_long_str(s):
    s = s.split(',')
    s = [x.strip() for x in s]
    return s
good_lecture = parse_long_str("사랑, 천사, 친절, 유명, 짱, 참여, 최고, 만족, 유익, 열정적, 갓, 답변, 명강의")
bad_lecture = parse_long_str("집중, 혼자, 부족, 자습, 당황, 최악, 인생, 불편, 복잡, 불만, 악명, 비추, 난이(도)")
#todo try weighted average of w2v
scores = {}
def add_to_dict(l,score):
    global scores
    print("Added " + str(len(l)))
    for i in range(len(l)):
        scores[l[i]] = score
add_to_dict(good_lecture,1.0)
add_to_dict(bad_lecture,-1.0)

Added 13
Added 13


In [20]:
def enlarge_train_data(data_train,y_train):
    x = list(data_train)
    y = list(y_train)
    for k,v in scores.items():
#         x.append(str(k)+"\n")
#         unit = "(%s,NNG) " % (str(k))
        unit = "%s " % (str(k))
        x.append(unit*1)
        
        if v > 0:
            y.append(int(mp['A']))
        else:
            y.append(int(mp['F']))
    
    return np.array(x),np.array(y)

# data_train,y_train = enlarge_train_data(data_train,y_train)

In [21]:
len(data_train)

432

In [22]:
len(y_train)

432

In [23]:
vectorizer = TfidfVectorizer(min_df=3, max_df = 0.8,max_features=200000)

In [24]:
# vectirizer.fit(..)
# X_train = vectorizer.transform(data_train)
X_train = vectorizer.fit_transform(data_train)
X_test = vectorizer.transform(data_test)

In [25]:
tfidf_features = vectorizer.get_feature_names()

<2x832 sparse matrix of type '<class 'numpy.float64'>'
	with 26 stored elements in Compressed Sparse Row format>

In [26]:
len(tfidf_features)

832

In [27]:
tfidf_features

['10',
 '100',
 '100명',
 '100점',
 '101',
 '10개',
 '10번',
 '10분',
 '10점',
 '11',
 '12',
 '15',
 '16',
 '17',
 '1개',
 '1번',
 '1시간',
 '1시그마',
 '1점',
 '1조',
 '20',
 '200',
 '20분',
 '20점',
 '21',
 '25',
 '2개',
 '2명',
 '2번',
 '2시간',
 '2인',
 '2인1조',
 '2주',
 '2학년',
 '30',
 '300',
 '300번대',
 '30분',
 '340',
 '3개',
 '3문제',
 '3번',
 '3시간',
 '3시간정도',
 '3일',
 '3학년',
 '3학점',
 '40',
 '4개',
 '4문제',
 '4번',
 '4시간',
 '4학점',
 '50',
 '5개',
 '5번',
 '5분',
 '60',
 '6개',
 '6번',
 '70',
 '7개',
 '80',
 '80점',
 '8개',
 '8번',
 '90',
 '90점',
 '98',
 '9번',
 '9학점',
 '가기',
 '가능',
 '가도',
 '가량',
 '가면',
 '가시길',
 '가을',
 '가지',
 '가차',
 '가치',
 '각각',
 '각오',
 '각종',
 '감사',
 '감안',
 '감점',
 '강성원',
 '강의',
 '강의평가',
 '강추',
 '개념',
 '개념적',
 '개론',
 '개발',
 '개선',
 '개인',
 '개인적',
 '거기',
 '거리',
 '거임',
 '거지',
 '걱정',
 '건강',
 '건의',
 '게다',
 '게임',
 '결과',
 '결국',
 '결론',
 '결석',
 '결정',
 '경우',
 '경향',
 '경험',
 '계산',
 '계열',
 '계획',
 '고등',
 '고등학교',
 '고려',
 '고민',
 '고사',
 '고생',
 '고요',
 '고통',
 '공개',
 '공부',
 '공부량',
 '공유',
 '공지',
 '공학',
 '과목',
 '과목후기',
 '과정',
 '과제'

In [1099]:
print(X_train.shape)

(372, 917)


In [1100]:
print(X_test.shape)

(42, 917)


In [1115]:
clf = LinearSVC()
clf.fit(X_train,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [1116]:
predictions = clf.predict(X_test)
accuracy_score(y_test,predictions)

0.8095238095238095

In [1117]:
mean_squared_error(y_test,predictions)

0.19047619047619047

In [1118]:
predictions

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [1119]:
y_test

array([1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1])

In [1120]:
for i in range(len(predictions)):
    if predictions[i]!= y_test[i]:
#         print("pred",predictions[i],"real",y_test[i])
#         print(data_test[i])
        pass
        

In [1114]:
micro = f1_score(y_test, predictions, average='micro')  
macro = f1_score(y_test, predictions, average='macro')  
print(micro,macro)
f1_score(y_test, predictions, average='weighted')  

0.7380952380952381 0.49945828819068255


0.6475777743383377

In [1122]:
roc_auc_score(y_test,predictions)

0.6666666666666667

In [1131]:
LinearSVC().get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': True,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'loss': 'squared_hinge',
 'max_iter': 1000,
 'multi_class': 'ovr',
 'penalty': 'l2',
 'random_state': None,
 'tol': 0.0001,
 'verbose': 0}

In [1138]:
params = {'penalty':['l2'],
          'C': [10**i for i in range(-2,2)]}

gs2 = grid_search.GridSearchCV(
    estimator=LinearSVC(),
    param_grid=params,
    cv=5,
    scoring='f1'
)

#fit the grid search object to our new dataset
print ('Fitting grid search...')
gs2.fit(X_train, y_train)
print ("Grid search fitted.")

Fitting grid search...
Grid search fitted.


In [1139]:
print (gs2.best_estimator_)
gs2.grid_scores_

LinearSVC(C=0.1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)


[mean: 0.79612, std: 0.00124, params: {'C': 0.01, 'penalty': 'l2'},
 mean: 0.80401, std: 0.00920, params: {'C': 0.1, 'penalty': 'l2'},
 mean: 0.76329, std: 0.02554, params: {'C': 1, 'penalty': 'l2'},
 mean: 0.74119, std: 0.03044, params: {'C': 10, 'penalty': 'l2'}]

In [ ]:
import gensim.downloader as api
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

In [41]:
dataset = api.load("text8")
dct = Dictionary(dataset)  # fit dictionary
corpus = [dct.doc2bow(line) for line in dataset]  # convert corpus to BoW format

model = TfidfModel(corpus)  # fit model
vector = model[corpus[0]]  # apply model to the first corpus document

In [43]:
opedataset.fn

'C:\\Users\\kvtor\\gensim-data\\text8\\text8.gz'

In [54]:
lls = [l.split() for l in concat_nouns_list]
dataset = lls

In [55]:
lls[:2]

[['1',
  '비현',
  '비현실적',
  '실적',
  '목표',
  '랜덤',
  '매칭',
  '학부',
  '명이',
  '3',
  '3달',
  '달',
  '동안',
  '구현',
  '가능',
  '특허',
  '수',
  '독창성',
  '신규',
  '실용성',
  '프로젝트',
  '주제',
  '2',
  '2주',
  '주',
  '안',
  '생각',
  '아이디어',
  '자퇴',
  '게이츠',
  '거',
  '개발',
  '시간',
  '내고',
  '명세서',
  '작성',
  '실제',
  '1.5',
  '1.5개월',
  '개월',
  '정도',
  '한국어',
  '수업',
  '전산',
  '전산필수',
  '필수',
  '외국인',
  '학생',
  '일부',
  '교수님',
  '강의',
  '4',
  '쓸모',
  '이것',
  '평가',
  '항목',
  '저희',
  '변리사',
  '도움',
  '소리',
  '정상',
  '저',
  '결론',
  '전산필수라',
  '필',
  '수라',
  '어자',
  '어자피',
  '피',
  '과목',
  '강의계획서',
  '계획서',
  '나',
  '단어',
  '마음',
  '준비'],
 ['시스템',
  '시스템쪽',
  '쪽',
  '강의',
  '영어',
  '유창',
  '컴퓨터',
  '아키텍쳐',
  '개념',
  '정도',
  '동시',
  '프로젝트',
  '수업',
  '수업시간',
  '시간',
  '내용',
  '구현',
  '방식',
  '수업간',
  '간',
  '상호',
  '시너지',
  '난이도',
  '학생',
  '의견',
  '반영',
  '조절',
  '신경',
  '주신',
  '느낌',
  '성적',
  '듯']]

In [56]:
dct = Dictionary(lls)  # fit dictionary
# dic = Dictionary()

In [57]:
corpus = [dct.doc2bow(line) for line in dataset]  # convert corpus to BoW format

In [64]:
model = TfidfModel(corpus)  # fit model
vector = model[corpus[4]]  # apply model to the first corpus document

In [70]:
for i in range(len(f)):
    vector = model[corpus[i]]
    
    for k,v in vector:
        print(dct[k],v)
    break

1 0.03744559178675608
1.5 0.12316023027288849
1.5개월 0.1665680503396307
2 0.035080102553360366
2주 0.09738932693015882
3 0.03677961239356559
3달 0.1665680503396307
4 0.04838130811363367
가능 0.08861242606643441
강의 0.023668218586705144
강의계획서 0.13693760347293943
개발 0.099548142393312
개월 0.1665680503396307
거 0.04318445816868635
게이츠 0.1665680503396307
결론 0.1140853065421441
계획서 0.13693760347293943
과목 0.0308931401975717
교수님 0.012822555081520361
구현 0.0638993365395059
나 0.07869459639034511
내고 0.14787331979981702
단어 0.14787331979981702
달 0.13693760347293943
도움 0.05659362366990308
독창성 0.1665680503396307
동안 0.08445485967545281
랜덤 0.10730715660624812
마음 0.0953905760023304
매칭 0.1291785892600033
명세서 0.14787331979981702
명이 0.1291785892600033
목표 0.12316023027288849
변리사 0.1665680503396307
비현 0.1665680503396307
비현실적 0.1665680503396307
생각 0.03387225318573107
소리 0.099548142393312
수 0.025051639516586094
수라 0.13693760347293943
수업 0.012286693844738171
시간 0.03290545451268147
신규 0.1665680503396307
실용성 0.166568050339

In [68]:
for k,v in dct.items():
    print(k,v)
    

0 1
1 1.5
2 1.5개월
3 2
4 2주
5 3
6 3달
7 4
8 가능
9 강의
10 강의계획서
11 개발
12 개월
13 거
14 게이츠
15 결론
16 계획서
17 과목
18 교수님
19 구현
20 나
21 내고
22 단어
23 달
24 도움
25 독창성
26 동안
27 랜덤
28 마음
29 매칭
30 명세서
31 명이
32 목표
33 변리사
34 비현
35 비현실적
36 생각
37 소리
38 수
39 수라
40 수업
41 시간
42 신규
43 실용성
44 실적
45 실제
46 쓸모
47 아이디어
48 안
49 어자
50 어자피
51 외국인
52 이것
53 일부
54 자퇴
55 작성
56 저
57 저희
58 전산
59 전산필수
60 전산필수라
61 정도
62 정상
63 주
64 주제
65 준비
66 특허
67 평가
68 프로젝트
69 피
70 필
71 필수
72 학부
73 학생
74 한국어
75 항목
76 간
77 개념
78 난이도
79 내용
80 느낌
81 동시
82 듯
83 반영
84 방식
85 상호
86 성적
87 수업간
88 수업시간
89 시너지
90 시스템
91 시스템쪽
92 신경
93 아키텍쳐
94 영어
95 유창
96 의견
97 조절
98 주신
99 쪽
100 컴퓨터
101 15
102 15분
103 7
104 7개
105 개
106 개인적
107 나중
108 눈
109 분
110 불성실
111 수업진도
112 실패
113 안주
114 입장
115 진도
116 처음
117 플젝
118 플젝들입니다
119 하루
120 학점
121 결정
122 교
123 꺼
124 도로
125 류
126 류석영교수님
127 번
128 석영
129 시대
130 시험
131 입문
132 조
133 조교
134 최고
135 추천
136 출석
137 친절하게
138 코딩
139 코딩입문
140 태
141 태도로
142 한
143 100
144 15가을
145 300
146 300점
147 320
148 400
149 400점
150 5
151 5개
152 5주


1666 벼락
1667 복잡
1668 불안
1669 시연
1670 윗
1671 유익하다
1672 책정
1673 편중
1674 하향
1675 강의실
1676 과학
1677 김동수
1678 드랍
1679 붙이기
1680 수리
1681 수리과학
1682 이산수학
1683 저쪽
1684 적당
1685 창피
1686 치명적
1687 동의
1688 보람
1689 수학적
1690 실무적
1691 여하
1692 공유
1693 모르고
1694 문제점
1695 방관
1696 음성적
1697 타과목
1698 터
1699 아드님
1700 요리
1701 바람
1702 분만
1703 옛날
1704 옛날분
1705 인
1706 주의바람
1707 별루
1708 별루임
1709 리딩
1710 리딩리스펀스
1711 리스
1712 펀스
1713 개략적
1714 걸
1715 발
1716 뻔
1717 성취감
1718 송교수님
1719 씨
1720 씨발
1721 음
1722 자살
1723 진심
1724 10시
1725 3줄
1726 목
1727 목소리
1728 세
1729 세서
1730 세장
1731 스케쥴
1732 스케쥴링
1733 실험
1734 실험날
1735 안내
1736 연관
1737 오류
1738 원인
1739 인터넷
1740 잠깐
1741 장비
1742 조사
1743 카피
1744 텍스트
1745 텍스트북
1746 파이널
1747 파트나
1748 프로
1749 프리랩
1750 거랑
1751 금
1752 냄새
1753 막했
1754 벤치
1755 벤치마크
1756 분석
1757 의심
1758 정책
1759 좌우
1760 89
1761 관리
1762 대학
1763 만족도
1764 밖
1765 시간관리
1766 신기
1767 얼마
1768 에너지
1769 영혼
1770 자연
1771 파악
1772 납득
1773 데이터구조
1774 문항
1775 설명기법
1776 셋
1777 유사
1778 클라우드
1779 특징
1780 1장
1781 갯수
1782 높으므
1783 마감
1784 상대평가
178

3166 신입생
3167 십상
3168 안정적
3169 양이
3170 입
3171 재학생
3172 적정선
3173 진학
3174 채로
3175 커뮤니케이션
3176 필수적인
3177 감당
3178 문의
3179 문의하면
3180 미출로
3181 여가
3182 여가시간
3183 차치
3184 출로
3185 할애
3186 1회
3187 85
3188 90대
3189 개판
3190 괴랄
3191 랄
3192 중중하
3193 중하
3194 직관적
3195 출석체크용도
3196 회
3197 계정
3198 그중
3199 근본
3200 근본적인
3201 나무
3202 뉴럴넷
3203 던
3204 마이너
3205 모델
3206 모델링
3207 복잡한것들이
3208 봇
3209 생성
3210 수풀
3211 순수머신러닝
3212 스크래치
3213 실러
3214 실러버스
3215 아웃풋
3216 연구
3217 오브젝트
3218 융합
3219 이미지
3220 인풋
3221 인풋아웃풋
3222 적극
3223 표현력
3224 걔네
3225 관련프로젝트
3226 교수진
3227 극복
3228 달정도
3229 문화
3230 문화충격
3231 비즈니스
3232 얘네
3233 요거
3234 인턴하다
3235 재주
3236 충격
3237 캐바캐
3238 폐지
3239 교양수업
3240 근래
3241 기록
3242 나이
3243 메
3244 메소드
3245 반쯤
3246 배정
3247 불완전
3248 산업
3249 소드
3250 스틱
3251 영국
3252 왕좌
3253 유명세
3254 이동거리
3255 인문계
3256 조가
3257 체어
3258 최단
3259 최단거리
3260 최단루트
3261 테스팅
3262 학회의
3263 현실적
3264 회의
3265 휴
3266 휴리스틱
3267 492
3268 ㅁ
3269 선형회귀
3270 스케줄링
3271 안정감
3272 참
3273 회귀
3274 거만
3275 그레이
3276 그레이딩
3277 나이아가라
3278 샘플링
3279 응교수님
3280 